In [ ]:
import numpy as np
#import pylab as plt
import pyccl as ccl
import matplotlib.pyplot as plt
from astropy.cosmology import Planck13
from astropy.cosmology import FlatLambdaCDM as FLCDM
%matplotlib inline

In [ ]:
def bins_labels(bins, **kwargs):
    bin_w = 0.025
    plt.xticks(np.arange(min(bins)+bin_w, max(bins)+bin_w, 2*bin_w), bins, **kwargs)


GCdata = np.loadtxt('cat.txt', usecols=(0,1))
#print(GCdata.shape)

M = GCdata[:,0]
z = GCdata[:,1]

#print(len(M), len(z))
#plt.scatter(M, 1+z)

In [ ]:
values_omegam = np.arange(0.15, 0.45, 0.1)
values_omegam = np.append(values_omegam, 0.32)
values_omegam = np.sort(values_omegam)

for r in values_omegam:
    cosmo = ccl.Cosmology(Omega_c=0.86*r, Omega_b=0.14*r, h=0.67, A_s=2.1e-9, n_s=0.96)
    cs = ccl.halos.MassFuncPress74(cosmo)
    
    cosmo_vol = FLCDM(H0 = 67.3, Om0 = r)

    z_bin = np.arange(0.05, 2.15, 0.1)
    #print(z_bin)
    M_bin = np.arange(14.025, 15.525, 0.05)
    #print(M_bin)

    bins = np.round(np.arange(14,15.6,0.05),3)

    i = 0
    k = 0
    log_l = []
    for red in z_bin:
        M_z = M[(z>=red-0.05) & (z<red+0.05)]
        plt.figure()
        n_obs, bins_hist, patches = plt.hist(M_z, bins, edgecolor = 'r', facecolor = 'red', alpha = 0.2)

        N = []
        log_p = []
        j = 0
        for mass in M_bin:
            ind_exp = np.arange(mass-0.025, mass+0.025,0.005)
            masses = 10**ind_exp
            nm = cs.get_mass_function(cosmo, masses, 1/(1+red))
            v_min = cosmo_vol.comoving_volume(red-0.05).value 
            v_max = cosmo_vol.comoving_volume(red+0.05).value 
            v_shell = (v_max-v_min)*0.36
            #print(v_max, v_min, v_shell)
            Nm = nm*v_shell
            k = np.trapz(Nm, x = ind_exp)
            N.append(k)
            log_p.append(n_obs[j]*np.log(k)-k)
            j += 1
        log_l.append(np.sum(log_p))  
        #bins_labels(bins, fontsize = 10)
        #plt.scatter(M_bin, N, marker = '.')
        #plt.savefig('mass_distr_%d.png' %i, bbox_inches = 'tight')
        plt.close()
        i += 1
    #print(log_l)
    plt.scatter(z_bin, log_l, label='Om = %.2f'%r) 
    plt.xlabel('z')
    plt.ylabel('Log(L)')
    plt.legend()
    plt.savefig('likel_om.png', bbox_inches = 'tight')